# Topological Data Analysis of pattern formation of human induced pluripotent stem cell colonies
# Iryna Hartsock, February 2023
------------------------------------------------------------------------

#### Dependencies

*Install the following libraries before running the computations.*
*For plotting persistence diagrams and persistence landscape computations  we use 'tda-tools' package. Since it is not on CRAN, follow this link https://github.com/jjbouza/tda-tools for installation instructions.*

In [ ]:
install.packages(c("Rcpp", "TDA", "kernlab", "ggplot2"))

------------------------------------------------------------------------

## Load libraries

In [ ]:
library("Rcpp")
library("TDA")
library("tdatools")
library("kernlab")
library("ggplot2")
library("ggpubr")

## Functions
*Pull generalized functions for TDA computations and for processing the CSV files from the discretization pipeline.*

In [ ]:
# get helper functions
source("utilities.R")

## Data parameters

In [ ]:
#Doxycycline concentrations
concentrations <- c(0,5,15,25)
#biological markers
groups <- c("Gata6","HA") 
nconcentrations <- length(concentrations)
ngroups <- length(groups)
#percentile
percentile <- 75
#number of images per concentration
nimages <- 15 
#number of patches per image
npatches <- 16 
#number of patches (samples) per concentration 
nsamples <- nimages*npatches 
#number of wells
nwells <- 3
#number of positions
npositions <- 5

## Data files 

In [ ]:
#specify the working directory path
working_dir_path <- "~/Documents/GitHub/TDA-Microscopy-Pipeline/TDA"

# specify location path of discretized CSVs
csv_dir_path <- "~/Documents/GitHub/TDA-Microscopy-Pipeline/tda-old/data/"

files <- list()
for (j in groups){
  for (c in concentrations){ 
    for (w in 1:nwells){
      for (p in 1:npositions){
        files <- append(files, paste0(percentile,"/Nanog_", j,"/", p, "/", c,"/", percentile, "_Nanog_",j, "_",c,"_", w, "_", p, ".csv"))
      }
    }
  }
}

## Pick a cell type: R+G-, R-G+, R+G+, or R-G-

In [ ]:
#choose a cell type
cell_type <- "R+G-" 

if (cell_type == "R+G-"){
  type <- 1
} else if (cell_type == "R-G+"){
  type <- 10
} else if (cell_type == "R+G+"){
  type <- 11
} else if (cell_type == "R-G-"){
  type <- 0
}

## Computation of persistence diagrams for every patch
*Since persistence computations might take long time to compute for a large data set, we recommend saving them*

In [ ]:
#persistence homology computation
PH <- list()
for (i in 1:length(files)){
  print(sprintf("Processing file %s", files[[i]]))
  PH[[i]] <- list()
  for (k in 1:sqrt(npatches)){ #y-axis
    for (j in 1:sqrt(npatches)){ #x-axis
      all_cells <- read.csv(file=paste0(csv_dir_path,files[[i]]), header=TRUE)
      #every image is ~2850x2850 pixels
      all_cells <- all_cells[which( (2850/sqrt(npatches))*(j-1) <= all_cells[,1] & all_cells[,1] < (2850/sqrt(npatches))*j & (2850/sqrt(npatches))*(k-1) <= all_cells[,2] & all_cells[,2] < (2850/sqrt(npatches))*k),]
      cells <- all_cells[ which(all_cells[,8]== type), ]               
      #compute persistence homology using the Delaunay complex filtration (also known as Alpha complex filtration)
      PH[[i]][[j+(k-1)*sqrt(npatches)]] <-  alphaComplexDiag(cells[,1:2], maxdimension = 1, library = c("GUDHI", "Dionysus"), location = TRUE)
    }
  } 
}

#create a list of persistence diagrams
PDs <-list() 
max_birth <- list()
max_death <- list()
for (i in 1:length(files)){
  PDs[[i]] <-list()
  max_birth[[i]] <- list()
  max_death[[i]] <- list()
  for (k in 1:npatches){
    #convert from gudhi to tdatools diagram data structure
    PDs[[i]][[k]] <- gudhi2tdatools(PH[[i]][[k]]$diagram) 
    #birth and death values have been squared, so take the square root
    #homology in degree 0
    PDs[[i]][[k]]$pairs[[1]] <- sqrt(PDs[[i]][[k]]$pairs[[1]]) 
    #homology in degree 1
    PDs[[i]][[k]]$pairs[[2]] <- sqrt(PDs[[i]][[k]]$pairs[[2]])
    #max birth of 1-degree persistence diagrams 
    max_birth[[i]][[k]] <- max(PDs[[i]][[k]]$pairs[[2]][,1])
    #max death of 1-degree persistence diagrams 
    max_death[[i]][[k]] <- max(PDs[[i]][[k]]$pairs[[2]][,2])
  }
}

max_birth <- max(unlist(max_birth))
max_death <- max(unlist(max_death))

#free memory by removing some data from the workspace
remove(list=c("PH"))

#create a folder for saved computations
dir.create("saved_computations")

#save the list of persistence diagrams, the max birth and max death values 
#separately for each cell type
save_filename <-  "persistence-diagrams-red.RData" 
save_file_location <- paste0(working_dir_path, "/saved_computations/") 
save(PDs, max_birth, max_death, file=paste0(save_file_location,save_filename)) 

In [ ]:
max_birth <- list()
max_death <- list()
for (i in 1:length(files)){
  max_birth[[i]] <- list()
  max_death[[i]] <- list()
  for (k in 1:npatches){
    #max birth of 1-degree persistence diagrams 
    max_birth[[i]][[k]] <- max(PDs[[i]][[k]]$pairs[[2]][,1])
    #max death of 1-degree persistence diagrams 
    max_death[[i]][[k]] <- max(PDs[[i]][[k]]$pairs[[2]][,2])
  }
}

max_birth <- max(unlist(max_birth))
max_death <- max(unlist(max_death))

## Plot persistence diagrams

In [ ]:
par(pty="s") #makes a square plot 

#load saved persistence diagrams
load(paste0(save_PD_file_location,save_PD_filename))

max_radius <- max(max_birth, max_death)
for (i in 1:length(files)){
  plot_diagram(PDs[[i]]$pairs[[2]], max_radius)
}

## Compute and plot persistence landscapes

In [ ]:
#load saved persistence diagrams
load(paste0(paste0(working_dir_path, "/saved_computations/"), "persistence-diagrams-red.RData"))

max_x <- (max_death+max_birth)/2 + 5
dx <- 0.3 #discretization step

PLs <- list()
for (i in 1:length(files)){ 
  PLs[[i]] <- list()
  for (k in 1:npatches){
    PLs[[i]][[k]] <- landscape0(PDs[[i]][[k]]$pairs[[2]], degree=1, exact=FALSE, dx=dx, min_x=0, max_x=max_x)
    plot_landscape(PLs[[i]][[k]], max_x, 0, max_death/2)
  }
}

## Compute and plot average persistence landscapes for each Dox concentration

In [ ]:
par(pty="s")

PLs_list <- list()
for (j in 1:ngroups){
  PLs_list[[j]] <- list()
  for (c in 1:nconcentrations){
    PLs_list[[j]][[c]] <- list()
    for (i in 1:nimages){
      for (k in 1:npatches){
        PLs_list[[j]][[c]] <- append(PLs_list[[j]][[c]],PLs[[nimages*(c-1)+(j-1)*nimages*nconcentrations+i]][[k]])
      }
    }
  }
}

PLs_avg <- list()
for (j in 1:ngroups){
  if (j == 1){
    group <- "Gata6" 
  } else {
    group <- "HA" 
  }
  PLs_avg[[j]] <- list()
  for (c in 1:nconcentrations){
    if (c==1){
      concentration <- 0
    } else if (c==2){
      concentration <- 5
    } else if (c==3){
      concentration <- 15
    } else {concentration <- 25 }
    PL_sum <- PLs_list[[j]][[c]][[1]]
    for (i in 2:length(PLs_list[[j]][[c]])){
      PL_sum <- PLsum(PL_sum, PLs_list[[j]][[c]][[i]])
    }
    PLs_avg[[j]][[c]]<- PLscale(1/(length(PLs_list[[j]][[c]])), PL_sum)
    plot_landscape(PLs_avg[[j]][[c]], max_x, 0, max_death/2)
    title(main=sprintf("Average PL for %s concentration %1.0f ng/ml",group, concentration)) 
  }
}

## Plot the difference between pan-GATA6 and HA average persistence landscape for each Dox concentration

In [ ]:
par(pty="s")
for (c in 1:nconcentrations){
   if (c==1){
      concentration <- 0
    } else if (c==2){
      concentration <- 5
    } else if (c==3){
      concentration <- 15
    } else {concentration <- 25 }
    plot_landscape(PLsum(PLs_avg[[1]][[c]],PLscale(-1,PLs_avg[[2]][[c]])), max_x, -max_death/2, max_death/2)
    title(main=sprintf("Gata6 vs HA concentration %1.0f ng/ml",concentration)) 
}

## Plot the difference between average persistence landscapes of distinct Dox concentrations

In [ ]:
for (j in 1:ngroups){
  par(pty="s")
  if (j == 1){
    group <- "Gata6" 
  } else {
    group <- "HA"
  }
  plot_landscape(PLsum(PLs_avg[[j]][[1]],PLscale(-1,PLs_avg[[j]][[2]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 0 vs 5 ng/ml",group)) 
  plot_landscape(PLsum(PLs_avg[[j]][[1]],PLscale(-1,PLs_avg[[j]][[3]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 0 vs 15 ng/ml",group)) 
  plot_landscape(PLsum(PLs_avg[[j]][[1]],PLscale(-1,PLs_avg[[j]][[4]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 0 vs 25 ng/ml",group)) 
  plot_landscape(PLsum(PLs_avg[[j]][[2]],PLscale(-1,PLs_avg[[j]][[3]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 5 vs 15 ng/ml",group)) 
  plot_landscape(PLsum(PLs_avg[[j]][[2]],PLscale(-1,PLs_avg[[j]][[4]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 5 vs 25 ng/ml",group)) 
  plot_landscape(PLsum(PLs_avg[[j]][[3]],PLscale(-1,PLs_avg[[j]][[4]])), max_x, -max_death/2, max_death/2)
  title(main=sprintf("%s concentration 15 vs 25 ng/ml",group)) 
} 

## Vectorize persistence landscapes 

In [ ]:
#choose a max PL depth 
PL_max_depth <- 30

vectorized_PLs <- vectorize_landscapes(PLs[[1]], PL_max_depth)
for (i in 2:length(files)){
  vectorized_PLs <- rbind(vectorized_PLs, vectorize_landscapes(PLs[[i]], PL_max_depth))
}

if (cell_type == "R+G-"){
  vec_red <- vectorized_PLs
  
  #save the vectorized persistence landscapes and their length
  save_filename <-  "PLs-red.RData" 
  save_file_location <- paste0(working_dir_path, "/saved_computations/") 
  save(vec_red, PL_length_red, file=paste0(save_file_location,save_filename)) 
} else if (cell_type == "R-G+"){
  vec_gr <- vectorized_PLs
  
  #save the vectorized persistence landscapes and their length
  save_filename <-  "PLs-gr.RData" 
  save_file_location <- paste0(working_dir_path, "/saved_computations/") 
  save(vec_gr, PL_length_gr, file=paste0(save_file_location,save_filename)) 
} else if (cell_type == "R+G+"){
  vec_pos <- vectorized_PLs
  
  #save the vectorized persistence landscapes and their length
  save_filename <-  "PLs-pos.RData" 
  save_file_location <- paste0(working_dir_path, "/saved_computations/") 
  save(vec_pos, PL_length_pos, file=paste0(save_file_location,save_filename)) 
} else if (cell_type == "R-G-"){
  vec_neg <- vectorized_PLs
  
  #save the vectorized persistence landscapes and their length
  save_filename <-  "PLs-neg.RData" 
  save_file_location <- paste0(working_dir_path, "/saved_computations/") 
  save(vec_neg, PL_length_neg, file=paste0(save_file_location,save_filename)) 
}

## Compute cell count vectors for every cell type

In [ ]:
cell_number_r <- list()
cell_number_gr <- list()
cell_number_pos <- list()
cell_number_neg <- list()
for (j in 1:ngroups){         #j=1 pan-Gata6, j=2 HA
  cell_number_r[[j]] <- list()
  cell_number_gr[[j]] <- list()
  cell_number_pos[[j]] <- list()
  cell_number_neg[[j]] <- list()
  for (c in 1:nconcentrations){             
    cell_number_r[[j]][[c]]<- list()
    cell_number_gr[[j]][[c]]<- list()
    cell_number_pos[[j]][[c]]<- list()
    cell_number_neg[[j]][[c]]<- list()
    for (i in 1:nimages){             
      all_cells <- read.csv(file=paste0(csv_dir_path,files[[i+(c-1)*nimages+(j-1)*nimages*nconcentrations]]), header=TRUE)
      cell_number_r[[j]][[c]][[i]] <- list()
      cell_number_gr[[j]][[c]][[i]] <- list()
      cell_number_pos[[j]][[c]][[i]] <- list()
      cell_number_neg[[j]][[c]][[i]] <- list()
      for (k in 1:sqrt(npatches)){
        for (r in 1:sqrt(npatches)){
          #every image is ~2850x2850 
          patch <- all_cells[which( ((2850/sqrt(npatches))*(r-1)) <= all_cells[,1] & all_cells[,1] < ((2850/sqrt(npatches))*r) & ((2850/sqrt(npatches))*(k-1)) <= all_cells[,2] & all_cells[,2] < ((2850/sqrt(npatches))*k)),]
          #R+G- cell type
          red_patch <- patch[ which((patch[,8]==1)), ] 
          cell_number_r[[j]][[c]][[i]][[r+(k-1)*sqrt(npatches)]] <- nrow(red_patch)
          #R-G+ cell type
          green_patch <- patch[ which((patch[,8]==10)), ] 
          cell_number_gr[[j]][[c]][[i]][[r+(k-1)*sqrt(npatches)]] <- nrow(green_patch)
          #R+G+ cell type
          pos_patch <- patch[ which((patch[,8]==11)), ] 
          cell_number_pos[[j]][[c]][[i]][[r+(k-1)*sqrt(npatches)]] <- nrow(pos_patch)
          #R-G-cell type
          neg_patch <- patch[ which((patch[,8]==0)), ] 
          cell_number_neg[[j]][[c]][[i]][[r+(k-1)*sqrt(npatches)]] <- nrow(neg_patch)
        }
      }
    }
  }
}

#concatenate cell counts of each cell type into a single vector
counts <- list()
for (j in 1:ngroups){
  counts[[j]] <- cbind(matrix(unlist(cell_number_r[[j]]), nrow=length(unlist(cell_number_r[[j]]))), 
                       matrix(unlist(cell_number_gr[[j]]), nrow=length(unlist(cell_number_gr[[j]]))), 
                       matrix(unlist(cell_number_pos[[j]]), nrow=length(unlist(cell_number_pos[[j]]))), 
                       matrix(unlist(cell_number_neg[[j]]), nrow=length(unlist(cell_number_neg[[j]]))))
}

## Load saved vectorized persistence landscapes for every cell type

In [ ]:
load(paste0(paste0(working_dir_path, "/saved_computations/"), "PLs-red.RData"))
load(paste0(paste0(working_dir_path, "/saved_computations/"), "PLs-gr.RData"))
load(paste0(paste0(working_dir_path, "/saved_computations/"), "PLs-pos.RData"))
load(paste0(paste0(working_dir_path, "/saved_computations/"), "PLs-neg.RData"))

## Permutation test
*pan-GATA6 vs HA for 25 ng/ml Dox concentration*

In [ ]:
vectors <- vec_red
p_value <- permutation_test(vectors[(3*nsamples+1):(4*nsamples),], vectors[(7*nsamples+1):(8*nsamples),])
print(sprintf("p-value %f:", p_value))

## Permutation test
*one vs another Dox concentration for each each group*

In [ ]:
vectors <- vec_red
for (i in 1:2){
  if (i == 1){group <- "pan-GATA6"} else {group <- "HA"} 
  p_value <- permutation_test(vectors[(0*nsamples+1+(i-1)*4*nsamples):(1*nsamples+(i-1)*4*nsamples),], vectors[(1*nsamples+1+(i-1)*4*nsamples):(2*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 0 vs 5 ng/ml has p-value %f:", group, p_value))
  p_value <- permutation_test(vectors[(0*nsamples+1+(i-1)*4*nsamples):(1*nsamples+(i-1)*4*nsamples),], vectors[(2*nsamples+1+(i-1)*4*nsamples):(3*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 0 vs 15 ng/ml has p-value %f:", group, p_value))
  p_value <- permutation_test(vectors[(0*nsamples+1+(i-1)*4*nsamples):(1*nsamples+(i-1)*4*nsamples),], vectors[(3*nsamples+1+(i-1)*4*nsamples):(4*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 0 vs 25 ng/ml has p-value %f:", group, p_value))
  p_value <- permutation_test(vectors[(1*nsamples+1+(i-1)*4*nsamples):(2*nsamples+(i-1)*4*nsamples),], vectors[(2*nsamples+1+(i-1)*4*nsamples):(3*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 5 vs 15 ng/ml has p-value %f:", group, p_value))
  p_value <- permutation_test(vectors[(1*nsamples+1+(i-1)*4*nsamples):(2*nsamples+(i-1)*4*nsamples),], vectors[(3*nsamples+1+(i-1)*4*nsamples):(4*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 5 vs 25 ng/ml has p-value %f:", group, p_value))
  p_value <- permutation_test(vectors[(2*nsamples+1+(i-1)*4*nsamples):(3*nsamples+(i-1)*4*nsamples),], vectors[(3*nsamples+1+(i-1)*4*nsamples):(4*nsamples+(i-1)*4*nsamples),])
  print(sprintf("%s group 0 vs 15 ng/ml has p-value %f:", group, p_value))
}

## Pairwise Support Vector Machine (SVM) classification

In [ ]:
#number of patches per group
nall_patches <- nsamples*nconcentrations
cost <- 10
nrepeats <- 20
num_folds <- 5
ntest_data <- 2*nsamples/num_folds
data_labels <- matrix(c(rep(1,(nsamples)),rep(-1,(nsamples))))
for (i in 1:2){
  if (i == 1){group <- "pan-GATA6"} else {group <- "HA"}
  
  #cell count vectors
  #vectors <- counts[[i]]
  
  #concatenate PL vectors of all four types
  vectors <- cbind(vec_red[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_gr[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_pos[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_neg[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),]) 

  #normalize vectors
  vectors <- vectors[, colSums(abs(vectors)) != 0]
  vectors <- scale(vectors)

  #create a list of pairwise combinations of vectors of different Dox concentration
  list_vectors <-list(rbind(vectors[1:(nsamples),], vectors[(nsamples+1):(2*nsamples),]), rbind(vectors[1:(nsamples),], vectors[(2*nsamples+1):(3*nsamples),]),
rbind(vectors[1:(nsamples),], vectors[(3*nsamples+1):(4*nsamples),]), 
rbind(vectors[(nsamples+1):(2*nsamples),], vectors[(2*nsamples+1):(3*nsamples),]), rbind(vectors[(nsamples+1):(2*nsamples),], vectors[(3*nsamples+1):(4*nsamples),]), rbind(vectors[(2*nsamples+1):(3*nsamples),], vectors[(3*nsamples+1):(4*nsamples),]))

  accuracy <- list()
  s <- list()
  for (k in 1:length(list_vectors)){
    if (k == 1){classification <- "0 vs 5 ng/ml"} 
    else if (k == 2){classification <- "0 vs 15 ng/ml"}
    else if (k == 3){classification <- "0 vs 25 ng/ml"}
    else if (k == 4){classification <- "5 vs 15 ng/ml"}
    else if (k == 5){classification <- "5 vs 25 ng/ml"}
    else if (k == 6){classification <- "15 vs 25 ng/ml"}
    s[[k]] <- list()
    for (j in 1:nrepeats){
      rand_perm <- sample(2*nsamples)
      vectors_permuted <- list_vectors[[k]][rand_perm,]
      data_labels_permuted <- data_labels[rand_perm]
      svm_model <- list()
      predictions_list <- vector()
      predictions <- list()
      for (n in 1:num_folds){
        svm_model[[n]] <-  ksvm(vectors_permuted[-((ntest_data*(n-1)+1):(ntest_data*(n-1)+ntest_data)),],data_labels_permuted[-((ntest_data*(n-1)+1):(ntest_data*(n-1)+ntest_data))],type="C-svc",scaled = c(),kernel="vanilladot", C=cost)
        predictions[[n]] <- predict(svm_model[[n]],vectors_permuted[((ntest_data*(n-1) +   1)):(ntest_data*(n-1)+ntest_data),])
        predictions_list <- c(predictions_list, predictions[[n]])
    }
    s[[k]][[j]] <- 0
    for (n in 1:(2*nsamples)){
      if (predictions_list[[n]] == data_labels_permuted[[n]]){
        s[[k]][[j]] <- s[[k]][[j]]+1
      }
     }
    }
    accuracy[[k]] <- sum(unlist(s[[k]]))/(nrepeats*2*nsamples)
    print(sprintf("For %s group the average accuracy for %s SVM classification is %#.2f%%", group, classification, 100*round(accuracy[[k]], 4)))
  }
}

## Multiclass SVM classification

In [ ]:
#number of patches per group
nall_patches <- nsamples*nconcentrations
cost <- 10
nrepeats <- 20
num_folds <- 10
ntest_data <- nall_patches/num_folds
data_labels <- matrix(c(rep(0,nsamples), rep(5,nsamples), rep(15,nsamples), rep(25,nsamples)))
for (i in 1:2){
  if (i == 1){group <- "pan-GATA6"} else {group <- "HA"}
  
  #cell count vectors
  #vectors <- counts[[i]]
  
  #concatenate PL vectors of all four types
  vectors <- cbind(vec_red[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_gr[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_pos[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_neg[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),]) 

  #normalize vectors
  vectors <- vectors[, colSums(abs(vectors)) != 0]
  vectors <- scale(vectors)
  
  accuracy <- list()
  total_accuracy <- 0
  for (k in 1:nrepeats){
    rand_perm <- sample(nall_patches)
    vectors_permuted <- vectors[rand_perm,]
    data_labels_permuted <- data_labels[rand_perm]
    svm_model <- list()
    predictions <- list()
    for (n in 1:num_folds){
      svm_model[[n]] <- ksvm(vectors_permuted[-((ntest_data*(n-1) +    1):(ntest_data*(n-1)+ntest_data)),],data_labels_permuted[-((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data))],type="C-svc",scaled = c(),kernel="vanilladot")
      predictions[[n]] <- predict(svm_model[[n]],vectors_permuted[((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data)),])
    }
    predictions <- unlist(predictions)
    s <- 0
    for (n in 1:nall_patches){
      if (predictions[[n]] == data_labels_permuted[[n]]){s <- s+1}
    }
    accuracy[[k]] <- s/nrow(vectors)
    print(sprintf("Accuracy is %#.2f %%", 100*round(accuracy[[k]], 4)))
    total_accuracy <- total_accuracy+accuracy[[k]]
  }
total_accuracy <- total_accuracy/nrepeats
print(sprintf("For %s group the average accuracy of multiclass SVM classification is %#.2f%%", group, 100*round(total_accuracy, 4)))
}

## Confusion matrix of an instance of multiclass SVM

In [ ]:
#number of patches per group
nall_patches <- nsamples*nconcentrations
cost <- 10
nrepeats <- 20
num_folds <- 10
ntest_data <- nall_patches/num_folds
data_labels <- matrix(c(rep(0,nsamples), rep(5,nsamples), rep(15,nsamples), rep(25,nsamples)))
for (i in 1:2){
  if (i == 1){group <- "pan-GATA6"} else {group <- "HA"}
  
  #cell count vectors
  #vectors <- counts[[i]]
  
  #concatenate PL vectors of all four types
  vectors <- cbind(vec_red[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_gr[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_pos[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_neg[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),]) 

  #normalize vectors
  vectors <- vectors[, colSums(abs(vectors)) != 0]
  vectors <- scale(vectors)

  accuracy <- list()
  total_accuracy <- 0
  rand_perm <- sample(nall_patches)
  vectors_permuted <- vectors[rand_perm,]
  data_labels_permuted <- data_labels[rand_perm]
  svm_model <- list()
  predictions <- list()
  for (n in 1:num_folds){
    svm_model[[n]] <- ksvm(vectors_permuted[-((ntest_data*(n-1) +    1):(ntest_data*(n-1)+ntest_data)),],data_labels_permuted[-((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data))],type="C-svc",scaled = c(),kernel="vanilladot")
    predictions[[n]] <- predict(svm_model[[n]],vectors_permuted[((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data)),])
  }
  predictions <- unlist(predictions)
  s <- 0
  for (n in 1:nall_patches){
    if (predictions[[n]] == data_labels_permuted[[n]]){s <- s+1}
  }
  print(sprintf("For %s group the accuracy of multiclass SVM classification is %#.2f%%", group, 100*round(s/nall_patches, 4)))
  print(table(unlist(predictions),data_labels_permuted))
}

## Support Vector Regression (SVR)

In [ ]:
#number of patches per group
nall_patches <- nsamples*nconcentrations
cost <- 10
nrepeats <- 20
num_folds <- 10
ntest_data <- nall_patches/num_folds
data_labels <- matrix(c(rep(0,nsamples), rep(5,nsamples), rep(15,nsamples), rep(25,nsamples)))
predictions <- list()
for (i in 1:2){
  if (i == 1){group <- "pan-GATA6"} else {group <- "HA"}
  
  #cell count vectors
  vectors <- counts[[i]]
  
  #concatenate PL vectors of all four types
  #vectors <- cbind(vec_red[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),],     vec_gr[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_pos[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),], vec_neg[(1+nall_patches*(1-i)):(nall_patches+nall_patches*(1-i)),]) 
  
  #normalize vectors
  vectors <- vectors[, colSums(abs(vectors)) != 0]
  vectors <- scale(vectors)
  
  predictions_list <- list()
  for (k in 1:nrepeats){
    rand_perm <- sample(nall_patches)
    perm_inv <- invPerm(rand_perm)
    vectors_permuted <- vectors[rand_perm,]
    data_labels_permuted <- data_labels[rand_perm]
    svr_model <- list()
    predictions_svr <- list()
    for (n in 1:num_folds){
      svr_model[[n]] <- ksvm(vectors_permuted[-((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data)),],data_labels_permuted[-((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data))],scaled  = FALSE ,kernel="vanilladot",C = cost,epsilon = 0.01)
      predictions_svr[[n]] <- predict(svr_model[[n]],vectors_permuted[((ntest_data*(n-1) + 1):(ntest_data*(n-1)+ntest_data)),])
    }
    predictions_list[[k]] <- unlist(predictions_svr)[perm_inv]
  }   
  prediction <- NULL
  for (k in 1:nrepeats){
    prediction <- rbind(prediction,predictions_list[[k]])
  }
  predictions[[i]] <- colSums(prediction)/nrepeats
}

## SVR: Dox concentration predictions plotted as a boxplot

In [ ]:
SVR_plot <- list()
for (i in 1:2){
  if (i == 1){
    group <- "Pan-GATA6"
    group_predictions <- predictions[[1]]
    } 
  else {
    group <- "HA"
    group_predictions <- predictions[[2]]
    }
  image_predictions <- vector()
  for (n in 1:(nimages*nconcentrations)){
    image_predictions[n] <- mean(sort(group_predictions[((n-1)*npatches+1):(n*npatches)]))
  }
  concentration <- c(rep("0",nimages),rep("5",nimages),rep("15",nimages),rep("25",nimages))
  value <- c(unlist(image_predictions))
  
  svr_data <- data.frame(concentration, value)
  svr_data <- svr_data %>% group_by(concentration) %>% filter(!is.na(value)) %>% 
  mutate(Outlier = ifelse(value > quantile(value, 0.75)+1.50*IQR(value),"Outlier","OK")) %>%
  mutate(Outlier = ifelse(value < quantile(value, 0.25)-1.50*IQR(value),"Outlier",Outlier))
  
  SVR_plot[[i]] <- ggplot(subset(svr_data, Outlier == "OK"), aes(x = concentration, y = value, fill=concentration))+geom_boxplot(outlier.colour = NA, alpha=0.4)+scale_x_discrete(limits=c("0", "5", "15", "25"))+xlab("Actual Dox concentration")+ylab("Predicted Dox concentration") +coord_cartesian(ylim=c(0,30))+scale_y_continuous(breaks = c(-5, 0, 5, 10,15,20,25,30)) +scale_fill_discrete(breaks=c('0', '5', '15', '25'))+guides(fill=guide_legend(title="Dox concentration")) +geom_jitter(color="black", size=0.9, alpha=0.7, width = 0.2)+geom_point(inherit.aes = FALSE, data = subset(svr_data, Outlier == "Outlier"), aes(x = concentration, y  = value), size = 0.9, color = "gray65", shape = 19)+theme_bw()+theme(panel.grid.minor = element_blank(),plot.title = element_text(size=18, hjust=0.5), aspect.ratio=1, text = element_text(size=18))+ggtitle(group)
}

ggarrange(SVR_plot[[1]], SVR_plot[[2]], nrow=1,common.legend = TRUE, legend = "none")